In [1]:
import json
import pandas as pd
import glob
import time
from tqdm import tqdm
import sys,os
import collections
import re
import csv
from datetime import datetime
import itertools
import numpy as np
import statistics

import scipy.stats as ss
import seaborn as sns

from copy import *
import requests

In [2]:
# folder = 'csv_he'
# folder = 'csv_pp_500'
folder = 'csv_b_500'
# selection_method = 1
selection_method = 2
# project = 'he'
# project = 'pp'
project = 'ba'

In [3]:
df_users = pd.read_csv(folder + '/df_users_metrics_clean_' + str(selection_method) + '.csv')
df_pageviews = pd.read_csv(folder + '/df_pageviews_metrics_' + str(selection_method) + '.csv')

In [4]:
urls = df_pageviews['url'].unique()
df_pv_product = df_pageviews[df_pageviews['pageview_product'] == 1]
unique_pv = df_pv_product['url'].unique()

In [5]:
len(unique_pv)

953

In [6]:
if project == 'he':
    root_dir = './pageviews_he'
    element = 'span'
    key = 'class'
    value = 'price-final-holder'
    
if project == 'pp':
    root_dir = './pageviews_pp_500'
    element = 'span'
    key = 'id'
    value = 'our_price_display'
    
if project == 'ba':
    root_dir = './pageviews_b_500'
    element = 'span'
    key = 'class'
    value = 'cena'

In [ ]:
# write data 
for url in unique_pv:
    data = requests.get(url).content
    filename = root_dir + '/pv_' + str(i) + ".txt"
    open(filename, 'wb').write(data)
    
    f = open(filename, "r", encoding='utf-8')
    data = f.read()
    try: 
        from BeautifulSoup import BeautifulSoup
    except ImportError:
        from bs4 import BeautifulSoup
    parsed_html = BeautifulSoup(data)
    price = parsed_html.body.find(element, attrs={key: value})
            
    if price is not None:
        price = re.sub('[^0-9,]','',price.text.strip())
        df_pv_product.loc[df_pv_product['url'] == url,'price'] = re.sub(',','.',price) 
    
    if project == 'he':
        rate = parsed_html.body.find('span', attrs={'class':'rate-average'})
        if rate is not None:
            rate = re.sub('[^0-9,]','',rate.text.strip())
            df_pv_product.loc[df_pv_product['url'] == url,'rate'] = re.sub(',','.',rate)

In [ ]:
listOfUrls = list()
for (dirpath, dirnames, filenames) in os.walk(root_dir):
    for file in filenames:        
        listOfUrls += [dirpath + "/" + file]

In [ ]:
for (filename,url) in zip(listOfUrls,unique_pv):
    with open(filename, 'r', encoding="utf8") as file:
        data = file.read()
        try: 
            from BeautifulSoup import BeautifulSoup
        except ImportError:
            from bs4 import BeautifulSoup
        parsed_html = BeautifulSoup(data)
        price = parsed_html.body.find(element, attrs={key: value})
        
        if price is not None:
            price = re.sub('[^0-9,]','',price.text.strip())
            df_pv_product.loc[df_pv_product['url'] == url,'price'] = re.sub(',','.',price)            
        
        if project == 'he':
            rate = parsed_html.body.find('span', attrs={'class':'rate-average'})
            if rate is not None:
                rate = re.sub('[^0-9,]','',rate.text.strip())
                df_pv_product.loc[df_pv_product['url'] == url,'rate'] = re.sub(',','.',rate)

In [8]:
df_pv_product['price'] = pd.to_numeric(df_pv_product['price'])
df_users_price = df_pv_product.groupby('user_id')[['price']].mean().reset_index()
df_users_price.rename(columns={'price' : "mean_price"}, inplace = True)

<ipython-input-8-436f60591576>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pv_product['price'] = pd.to_numeric(df_pv_product['price'])


In [9]:
df_users = pd.merge(
    df_users,
    df_users_price,
    on='user_id',
    how='left'
)

columns = ['mean_price']

In [10]:
if project == 'he':
    df_pv_product['rate'] = pd.to_numeric(df_pv_product['rate'])
    
    df_users_rate = df_pv_product.groupby('user_id')[['rate']].mean().reset_index()
    df_users_rate.rename(columns={'rate' : "mean_rate"}, inplace = True)
    
    df_users = pd.merge(
        df_users,
        df_users_rate,
        on='user_id',
        how='left'
    )
    columns = ['mean_price', 'mean_rate']

In [11]:
for column in columns:
    df_users[column] = df_users[column].fillna(0)
    df_users[column] = df_users[column].astype(int)

In [12]:
df_users.to_csv(folder + '/df_users_dom_' + str(selection_method) + '.csv', index=False, encoding='utf-8-sig')